# FORECASTNG STOCK MARKET TRENDS

EDS 6340 INTRODUCTION TO DATA SCIENCE

GROUP 16


| **Name**                | **ID**      |
|-------------------------|-------------|
| Sai Shashank Gandavarapu | 2402666     |
| Aakanksha Govindaraju    | 2384001     |
| Gowtham Chowdam          | 2397801     |
| Kiran Kulkarni           | 2330104     |
| Nagesh Vaka              | 2407326     |


## Importing Packages

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import BDay
import holidays

## Load the Datasets

In [ ]:
df1 = pd.read_csv('Processed_DJI.csv')
df2 = pd.read_csv('Processed_NASDAQ.csv')
df3 = pd.read_csv('Processed_NYSE.csv')
df4 = pd.read_csv('Processed_RUSSELL.csv')
df5 = pd.read_csv('Processed_S&P.csv')

# Data Pre-Processing Modules

### Data Cleaning

Evaluating the quality of the data is the first step in cleaning it. This entails looking for formatting errors, duplicate entries, and missing values. We are perfoming time-based interpolation, ffill, bfill and mean imputation.

In [ ]:
def FillMissingValues(df):
    # Convert the 'Date' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'])

    # Set the 'Date' column as the index
    df = df.set_index('Date')

    # Check for missing dates in the index
    idx = pd.date_range(start=df.index.min(), end=df.index.max())
    df = df.reindex(idx)
    # Fill missing values using time-based interpolation
    for col in df.columns:
        # Check if the column is numeric before interpolating
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].interpolate(method='time')
    # Fill missing values using linear interpolation
    df.interpolate(method='linear', inplace=True)
    # Fill missing values using forward fill
    df = df.ffill()
    # Fill missing values using backward fill
    df = df.bfill()

    # If any missing values remain after interpolation, fill with the mean of the column
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].mean())
    df.index.name = 'Date'
    df = df.reset_index()
    return df

### Outlier Handling

Another problem is outliers, or data points that differ greatly from other observations. To identify and manage outliers, we find the Z-score.

In [ ]:
def detect_and_handle_outliers(df, column):
    if pd.api.types.is_numeric_dtype(df[column]):
        z_scores = np.abs(stats.zscore(df[column]))
        threshold = 3
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_labels = df.index[outlier_indices]
        df.loc[outlier_labels, column] = df[column].median()
    return df

## Data Transformation

### Data Scaling and Normalization

Another step in the process is standardizing data formats. Accurate analysis requires that dates, currency values, and categorical variables adhere to consistent conventions.

In [ ]:
def scale_data(df):
  numerical_features = df.select_dtypes(include=['number']).columns
  scaler = MinMaxScaler()
  df[numerical_features] = scaler.fit_transform(df[numerical_features])
  return df

# Data Cleaning

Perform Data Cleaning and Pre-processing steps to each dataset one by one.

## DJI

In [ ]:
df1.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,10428.049805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
1,2010-01-04,10583.959961,NaN,0.014951,NaN,NaN,NaN,NaN,NaN,NaN,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
2,2010-01-05,10572.019531,NaN,-0.001128,0.014951,NaN,NaN,NaN,NaN,NaN,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
3,2010-01-06,10573.679688,0.515598,0.000157,-0.001128,0.014951,NaN,NaN,NaN,NaN,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
4,2010-01-07,10606.860352,9.776045,0.003138,0.000157,-0.001128,0.014951,NaN,NaN,NaN,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010-05-19,10444.370117,-0.854393,-0.006334,-0.010811,0.000534,-0.015097,-4.152921,-3.887610,-5.440333,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
96,2010-05-20,10068.009766,-13.676921,-0.036035,-0.006334,-0.010811,0.000534,-6.630286,-4.299399,-9.843996,...,-2.00,-2.19,-4.85,-3.60,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
97,2010-05-21,10193.389648,-1.115597,0.012453,-0.036035,-0.006334,-0.010811,-4.018494,-1.801853,-7.405301,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50
98,2010-05-24,10066.570313,7.754703,-0.012441,0.012453,-0.036035,-0.006334,-5.263210,-6.662587,-9.731674,...,-1.03,1.23,-1.20,-1.24,0.91,0.94,0.98,-1.30,1.25,1.27


In [ ]:
# Check for missing values in each column
missing_values = df1.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Volume       3
mom          1
mom1         2
mom2         3
mom3         4
ROC_5        5
ROC_10      10
ROC_15      15
ROC_20      20
EMA_10       9
EMA_20      19
EMA_50      49
EMA_200    199
Oil          1
Gold         1
GBP        478
JPY        479
CAD        483
CNY        479
AAPL         1
AMZN         1
GE           1
JNJ          1
JPM          1
MSFT         1
WFC          1
XOM          1
FCHI        40
FTSE        71
GDAXI       67
GSPC         1
HSI        170
IXIC         1
SSEC       238
RUT          1
NYSE         1
CTB3M        1
CTB6M        1
CTB1Y        1
CAC-F        4
DAX-F        4
FTSE-F       6
HSI-F       75
KOSPI-F     79
wheat-F      2
dtype: int64


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df1.select_dtypes(include=[np.number])))

outliers = (z_scores > 5).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')


Number of outlier rows: 53


In [ ]:
df1 = FillMissingValues(df1)

In [ ]:
us_holidays = holidays.US()
df1['Is_Business_Day'] = df1['Date'].apply(
    lambda x: x.weekday() < 5 and x not in us_holidays)
# Keep only business days
df1 = df1[df1['Is_Business_Day']].drop(columns=['Is_Business_Day'])


In [ ]:
# Check for missing values in each column
missing_values = df1.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df1.select_dtypes(include=[np.number])))

outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

while outliers.sum() > 40:
    # Loop through columns except 'Close'
    for col in df1.columns:
      if col != 'Close':
        df1 = detect_and_handle_outliers(df1, col)
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df1.select_dtypes(include=[np.number])))

        # Identify rows with Z-scores greater than 3 (typical threshold for outliers)
        outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

Number of outlier rows: 694
Number of outlier rows: 27


In [ ]:
df1["Name"] = df1["Name"].replace("DJI", 1)

In [ ]:
df1.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,10428.049805,0.515598,0.014951,0.014951,0.014951,0.014951,1.823357,1.741463,-2.216996,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
4,2010-01-04,10583.959961,0.515598,0.014951,0.014951,0.014951,0.014951,1.823357,1.741463,-2.216996,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
5,2010-01-05,10572.019531,0.515598,-0.001128,0.014951,0.014951,0.014951,1.823357,1.741463,-2.216996,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
6,2010-01-06,10573.679688,0.515598,0.000157,-0.001128,0.014951,0.014951,1.823357,1.741463,-2.216996,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
7,2010-01-07,10606.860352,-0.665845,0.003138,0.000157,-0.001128,0.014951,1.823357,1.741463,-2.216996,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2010-05-18,10510.950195,0.216584,-0.010811,0.000534,-0.015097,-0.010458,-2.207888,-3.805510,-4.376278,...,-1.52,0.11,-1.74,-1.39,0.00,1.08,1.11,-0.56,0.37,-0.17
139,2010-05-19,10444.370117,-0.854393,-0.006334,-0.010811,0.000534,-0.015097,-4.152921,-3.887610,-5.440333,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
140,2010-05-20,10068.009766,-0.665845,0.000540,-0.006334,-0.010811,0.000534,0.340396,-4.299399,0.888749,...,-2.00,-2.19,0.13,0.06,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
141,2010-05-21,10193.389648,-1.115597,0.012453,0.000535,-0.006334,-0.010811,-4.018494,-1.801853,0.888749,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50


In [ ]:
df1.to_csv('Cleaned&Processed_DJI.csv')

## NASDAQ

In [ ]:
df2.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,12/31/09,2269.149902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
1,1/4/10,2308.419922,0.560308,0.017306,NaN,NaN,NaN,NaN,NaN,NaN,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
2,1/5/10,2308.709961,0.225994,0.000126,0.017306,NaN,NaN,NaN,NaN,NaN,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
3,1/6/10,2301.090088,-0.048364,-0.003300,0.000126,0.017306,NaN,NaN,NaN,NaN,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
4,1/7/10,2300.050049,0.007416,-0.000452,-0.003300,0.000126,0.017306,NaN,NaN,NaN,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5/19/10,2298.370117,0.062583,-0.008152,-0.015704,0.003145,-0.019842,-5.222633,-4.325869,-7.013706,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
96,5/20/10,2204.010010,0.292598,-0.041055,-0.008152,-0.015704,0.003145,-7.949936,-4.984820,-12.257951,...,-2.00,-2.19,-4.85,-3.60,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
97,5/21/10,2229.040039,-0.008990,0.011357,-0.041055,-0.008152,-0.015704,-5.019923,-1.615431,-9.432425,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50
98,5/24/10,2213.550049,-0.365714,-0.006949,0.011357,-0.041055,-0.008152,-5.975624,-6.784938,-11.413350,...,-1.03,1.23,-1.20,-1.24,0.91,0.94,0.98,-1.30,1.25,1.27


In [ ]:
# Check for missing values in each column
missing_values = df2.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Volume       1
mom          1
mom1         2
mom2         3
mom3         4
ROC_5        5
ROC_10      10
ROC_15      15
ROC_20      20
EMA_10       9
EMA_20      19
EMA_50      49
EMA_200    199
Oil          1
Gold         1
GBP        478
JPY        479
CAD        483
CNY        479
AAPL         1
AMZN         1
GE           1
JNJ          1
JPM          1
MSFT         1
WFC          1
XOM          1
FCHI        40
FTSE        71
GDAXI       67
GSPC         1
HSI        170
DJI          1
SSEC       238
RUT          1
NYSE         1
CTB3M        1
CTB6M        1
CTB1Y        1
CAC-F        4
DAX-F        4
FTSE-F       6
HSI-F       75
KOSPI-F     79
wheat-F      2
dtype: int64


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df2.select_dtypes(include=[np.number])))

outliers = (z_scores > 5).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')


Number of outlier rows: 53


In [ ]:
df2 = FillMissingValues(df2)

In [ ]:
df2['Is_Business_Day'] = df2['Date'].apply(
    lambda x: x.weekday() < 5 and x not in us_holidays)
# Keep only business days
df2 = df2[df2['Is_Business_Day']].drop(columns=['Is_Business_Day'])


In [ ]:
# Check for missing values in each column
missing_values = df2.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df2.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

while outliers.sum() > 40:
    # Loop through columns except 'Close'
    for col in df2.columns:
      if col != 'Close':
        df2 = detect_and_handle_outliers(df2, col)
        z_scores = np.abs(stats.zscore(df2.select_dtypes(include=[np.number])))
        outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

Number of outlier rows: 695
Number of outlier rows: 6


In [ ]:
df2["Name"] = df2["Name"].replace("NASDAQ", 2)

In [ ]:
df2.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,2269.149902,0.000000,0.017306,0.017306,0.017306,0.017306,2.116212,0.830271,-2.571441,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
4,2010-01-04,2308.419922,0.000000,0.017306,0.017306,0.017306,0.017306,2.116212,0.830271,-2.571441,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
5,2010-01-05,2308.709961,0.225994,0.000126,0.017306,0.017306,0.017306,2.116212,0.830271,-2.571441,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
6,2010-01-06,2301.090088,-0.048364,-0.003300,0.000126,0.017306,0.017306,2.116212,0.830271,-2.571441,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
7,2010-01-07,2300.050049,0.007416,-0.000452,-0.003300,0.000126,0.017306,2.116212,0.830271,-2.571441,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2010-05-18,2317.260010,0.022933,-0.015704,0.003145,-0.019842,-0.012643,-2.443894,-4.413323,-6.239605,...,-1.52,0.11,-1.74,-1.39,0.00,1.08,1.11,-0.56,0.37,-0.17
139,2010-05-19,2298.370117,0.062583,-0.008152,-0.015704,0.003145,-0.019842,-5.222633,-4.325869,-7.013706,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
140,2010-05-20,2204.010010,0.292598,0.000957,-0.008152,-0.015704,0.003145,0.487818,-4.984820,1.136129,...,-2.00,-2.19,0.13,0.06,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
141,2010-05-21,2229.040039,-0.008990,0.011357,0.000957,-0.008152,-0.015704,-5.019923,-1.615431,1.136129,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50


In [ ]:
df2.to_csv('Cleaned&Processed_NASDAQ.csv')

## NYSE

In [ ]:
df3.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,7184.959961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
1,2010-01-04,7326.740234,0.921723,0.019733,NaN,NaN,NaN,NaN,NaN,NaN,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
2,2010-01-05,7354.870117,-0.375903,0.003839,0.019733,NaN,NaN,NaN,NaN,NaN,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
3,2010-01-06,7377.700195,0.996234,0.003104,0.003839,0.019733,NaN,NaN,NaN,NaN,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
4,2010-01-07,7393.930176,0.059932,0.002200,0.003104,0.003839,0.019733,NaN,NaN,NaN,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010-05-19,6927.209961,0.096415,-0.004598,-0.014811,-0.001951,-0.021665,-5.318901,-4.557855,-7.633755,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
96,2010-05-20,6653.000000,0.230981,-0.039584,-0.004598,-0.014811,-0.001951,-8.036223,-5.118711,-12.336991,...,-2.00,-2.19,-4.85,-3.60,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
97,2010-05-21,6775.450195,-0.345370,0.018405,-0.039584,-0.004598,-0.014811,-4.269643,-2.034793,-9.351248,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50
98,2010-05-24,6666.740234,-0.041835,-0.016045,0.018405,-0.039584,-0.004598,-5.621452,-8.141510,-11.618268,...,-1.03,1.23,-1.20,-1.24,0.91,0.94,0.98,-1.30,1.25,1.27


In [ ]:
# Check for missing values in each column
missing_values = df3.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Volume       1
mom          1
mom1         2
mom2         3
mom3         4
ROC_5        5
ROC_10      10
ROC_15      15
ROC_20      20
EMA_10       9
EMA_20      19
EMA_50      49
EMA_200    199
Oil          1
Gold         1
GBP        478
JPY        479
CAD        483
CNY        479
AAPL         1
AMZN         1
GE           1
JNJ          1
JPM          1
MSFT         1
WFC          1
XOM          1
FCHI        40
FTSE        71
GDAXI       67
DJI          1
HSI        170
IXIC         1
SSEC       238
S&P          1
RUT          1
CTB3M        1
CTB6M        1
CTB1Y        1
CAC-F        4
DAX-F        4
FTSE-F       6
HSI-F       75
KOSPI-F     79
wheat-F      2
dtype: int64


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df3.select_dtypes(include=[np.number])))

outliers = (z_scores > 5).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')


Number of outlier rows: 53


In [ ]:
df3 = FillMissingValues(df3)

In [ ]:
df3['Is_Business_Day'] = df3['Date'].apply(
    lambda x: x.weekday() < 5 and x not in us_holidays)
# Keep only business days
df3 = df3[df3['Is_Business_Day']].drop(columns=['Is_Business_Day'])


In [ ]:
# Check for missing values in each column
missing_values = df3.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df3.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

while outliers.sum() > 40:
    # Loop through columns except 'Close'
    for col in df3.columns:
      if col != 'Close':
        df3 = detect_and_handle_outliers(df3, col)
        z_scores = np.abs(stats.zscore(df3.select_dtypes(include=[np.number])))
        outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

Number of outlier rows: 719
Number of outlier rows: 6


In [ ]:
df3["Name"] = df3["Name"].replace("NYA", 3)

In [ ]:
df3.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,7184.959961,-0.001183,0.019733,0.019733,0.019733,0.019733,3.345741,2.391525,-1.556447,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
4,2010-01-04,7326.740234,-0.001183,0.019733,0.019733,0.019733,0.019733,3.345741,2.391525,-1.556447,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
5,2010-01-05,7354.870117,-0.001183,0.003839,0.019733,0.019733,0.019733,3.345741,2.391525,-1.556447,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
6,2010-01-06,7377.700195,-0.001183,0.003104,0.003839,0.019733,0.019733,3.345741,2.391525,-1.556447,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
7,2010-01-07,7393.930176,0.059932,0.002200,0.003104,0.003839,0.019733,3.345741,2.391525,-1.556447,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2010-05-18,6959.209961,0.041858,-0.014811,-0.001951,-0.021665,-0.011206,-3.634209,-5.152340,-6.751626,...,-1.52,0.11,-1.74,-1.39,0.00,1.08,1.11,-0.56,0.37,-0.17
139,2010-05-19,6927.209961,0.096415,-0.004598,-0.014811,-0.001951,-0.021665,0.277196,-4.557855,0.686600,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
140,2010-05-20,6653.000000,0.230981,0.000529,-0.004598,-0.014811,-0.001951,0.277196,-5.118711,0.686600,...,-2.00,-2.19,0.13,0.06,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
141,2010-05-21,6775.450195,-0.345370,0.018405,0.000517,-0.004598,-0.014811,-4.269643,-2.034793,0.686600,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50


In [ ]:
df3.to_csv('Cleaned&Processed_NYSE.csv')

## RUSSELL

In [ ]:
df4.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,625.390015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
1,2010-01-04,640.099976,0.921723,0.023521,NaN,NaN,NaN,NaN,NaN,NaN,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
2,2010-01-05,638.489990,-0.375903,-0.002515,0.023521,NaN,NaN,NaN,NaN,NaN,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
3,2010-01-06,637.950012,0.996234,-0.000846,-0.002515,0.023521,NaN,NaN,NaN,NaN,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
4,2010-01-07,641.969971,0.059932,0.006301,-0.000846,-0.002515,0.023521,NaN,NaN,NaN,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010-05-19,674.400024,0.096415,-0.012230,-0.018628,0.002493,-0.022357,-5.824519,-3.461306,-6.643225,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
96,2010-05-20,640.039978,0.230981,-0.050949,-0.012230,-0.018628,0.002493,-9.834472,-4.788540,-13.243150,...,-2.00,-2.19,-4.85,-3.60,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
97,2010-05-21,649.289978,-0.345370,0.014452,-0.050949,-0.012230,-0.018628,-6.439667,-0.568150,-9.392967,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50
98,2010-05-24,641.210022,-0.041835,-0.012444,0.014452,-0.050949,-0.012230,-7.833724,-7.018454,-12.501021,...,-1.03,1.23,-1.20,-1.24,0.91,0.94,0.98,-1.30,1.25,1.27


In [ ]:
# Check for missing values in each column
missing_values = df4.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Volume       1
mom          1
mom1         2
mom2         3
mom3         4
ROC_5        5
ROC_10      10
ROC_15      15
ROC_20      20
EMA_10       9
EMA_20      19
EMA_50      49
EMA_200    199
Oil          1
Gold         1
GBP        478
JPY        479
CAD        483
CNY        479
AAPL         1
AMZN         1
GE           1
JNJ          1
JPM          1
MSFT         1
WFC          1
XOM          1
FCHI        40
FTSE        71
GDAXI       67
DJI          1
HSI        170
IXIC         1
SSEC       238
S&P          1
NYSE         1
CTB3M        1
CTB6M        1
CTB1Y        1
CAC-F        4
DAX-F        4
FTSE-F       6
HSI-F       75
KOSPI-F     79
wheat-F      2
dtype: int64


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df4.select_dtypes(include=[np.number])))
outliers = (z_scores > 5).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')


Number of outlier rows: 53


In [ ]:
df4 = FillMissingValues(df4)

In [ ]:
df4['Is_Business_Day'] = df4['Date'].apply(
    lambda x: x.weekday() < 5 and x not in us_holidays)
# Keep only business days
df4 = df4[df4['Is_Business_Day']].drop(columns=['Is_Business_Day'])


In [ ]:
# Check for missing values in each column
missing_values = df4.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df4.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

while outliers.sum() > 40:
    # Loop through columns except 'Close'
    for col in df4.columns:
      if col != 'Close':
        df4 = detect_and_handle_outliers(df4, col)
        z_scores = np.abs(stats.zscore(df4.select_dtypes(include=[np.number])))
        outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

Number of outlier rows: 707
Number of outlier rows: 6


In [ ]:
df4["Name"] = df4["Name"].replace("RUT", 4)

In [ ]:
df4.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,625.390015,-0.001074,0.023521,0.023521,0.023521,0.023521,3.065284,2.009947,-1.164078,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
4,2010-01-04,640.099976,-0.001074,0.023521,0.023521,0.023521,0.023521,3.065284,2.009947,-1.164078,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
5,2010-01-05,638.489990,-0.001074,-0.002515,0.023521,0.023521,0.023521,3.065284,2.009947,-1.164078,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
6,2010-01-06,637.950012,-0.001074,-0.000846,-0.002515,0.023521,0.023521,3.065284,2.009947,-1.164078,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
7,2010-01-07,641.969971,0.059932,0.006301,-0.000846,-0.002515,0.023521,3.065284,2.009947,-1.164078,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2010-05-18,682.750000,0.041858,-0.018628,0.002493,-0.022357,-0.008742,-1.830388,-3.797381,-5.340582,...,-1.52,0.11,-1.74,-1.39,0.00,1.08,1.11,-0.56,0.37,-0.17
139,2010-05-19,674.400024,0.096415,-0.012230,-0.018628,0.002493,-0.022357,-5.824519,-3.461306,-6.643225,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
140,2010-05-20,640.039978,0.230981,0.001052,-0.012230,-0.018628,0.002493,0.416839,-4.788540,0.926836,...,-2.00,-2.19,0.13,0.06,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
141,2010-05-21,649.289978,-0.345370,0.014452,0.000959,-0.012230,-0.018628,-6.439667,-0.568150,-9.392967,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50


In [ ]:
df4.to_csv('Cleaned&Processed_RUSSELL.csv')

## S&P

In [ ]:
df5.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,1115.099976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
1,2010-01-04,1132.989990,0.921723,0.016043,NaN,NaN,NaN,NaN,NaN,NaN,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
2,2010-01-05,1136.520020,-0.375903,0.003116,0.016043,NaN,NaN,NaN,NaN,NaN,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
3,2010-01-06,1137.140015,0.996234,0.000546,0.003116,0.016043,NaN,NaN,NaN,NaN,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
4,2010-01-07,1141.689941,0.059932,0.004001,0.000546,0.003116,0.016043,NaN,NaN,NaN,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010-05-19,1115.050049,0.096415,-0.005130,-0.014196,0.001109,-0.018800,-4.832418,-4.358972,-6.405279,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
96,2010-05-20,1071.589966,0.230981,-0.038976,-0.005130,-0.014196,0.001109,-7.417229,-5.013523,-11.202544,...,-2.00,-2.19,-4.85,-3.60,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
97,2010-05-21,1087.689941,-0.345370,0.015024,-0.038976,-0.005130,-0.014196,-4.225672,-2.087540,-8.342533,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50
98,2010-05-24,1073.650024,-0.041835,-0.012908,0.015024,-0.038976,-0.005130,-5.566690,-7.422414,-10.697352,...,-1.03,1.23,-1.20,-1.24,0.91,0.94,0.98,-1.30,1.25,1.27


In [ ]:
# Check for missing values in each column
missing_values = df5.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Volume       1
mom          1
mom1         2
mom2         3
mom3         4
ROC_5        5
ROC_10      10
ROC_15      15
ROC_20      20
EMA_10       9
EMA_20      19
EMA_50      49
EMA_200    199
Oil          1
Gold         1
GBP        478
JPY        479
CAD        483
CNY        479
AAPL         1
AMZN         1
GE           1
JNJ          1
JPM          1
MSFT         1
WFC          1
XOM          1
FCHI        40
FTSE        71
GDAXI       67
DJI          1
HSI        170
IXIC         1
SSEC       238
RUT          1
NYSE         1
CTB3M        1
CTB6M        1
CTB1Y        1
CAC-F        4
DAX-F        4
FTSE-F       6
HSI-F       75
KOSPI-F     79
wheat-F      2
dtype: int64


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df5.select_dtypes(include=[np.number])))
outliers = (z_scores > 5).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')


Number of outlier rows: 53


In [ ]:
df5 = FillMissingValues(df5)

In [ ]:
df5['Is_Business_Day'] = df5['Date'].apply(
    lambda x: x.weekday() < 5 and x not in us_holidays)
# Keep only business days
df5 = df5[df5['Is_Business_Day']].drop(columns=['Is_Business_Day'])


In [ ]:
# Check for missing values in each column
missing_values = df5.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df5.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

while outliers.sum() > 40:
    # Loop through columns except 'Close'
    for col in df5.columns:
      if col != 'Close':
        df5 = detect_and_handle_outliers(df5, col)
        z_scores = np.abs(stats.zscore(df5.select_dtypes(include=[np.number])))
        outliers = (z_scores > 3).any(axis=1)
print(f'Number of outlier rows: {outliers.sum()}')

Number of outlier rows: 715
Number of outlier rows: 6


In [ ]:
df5["Name"] = df5["Name"].replace("S&P", 5)

In [ ]:
df5.head(100)

,Date,Close,Volume,mom,mom1,mom2,mom3,ROC_5,ROC_10,ROC_15,...,NZD,silver-F,RUSSELL-F,S&P-F,CHF,Dollar index-F,Dollar index,wheat-F,XAG,XAU
0,2009-12-31,1115.099976,-0.001074,0.016043,0.016043,0.016043,0.016043,2.679581,1.876967,-1.642897,...,0.03,0.26,-1.08,-1.00,-0.11,-0.08,-0.06,-0.48,0.30,0.39
4,2010-01-04,1132.989990,-0.001074,0.016043,0.016043,0.016043,0.016043,2.679581,1.876967,-1.642897,...,1.52,3.26,1.61,1.62,-0.57,-0.59,-0.42,3.12,3.91,2.10
5,2010-01-05,1136.520020,-0.001074,0.003116,0.016043,0.016043,0.016043,2.679581,1.876967,-1.642897,...,-0.07,1.96,-0.20,0.31,0.43,0.03,0.12,-0.90,1.42,-0.12
6,2010-01-06,1137.140015,-0.001074,0.000546,0.003116,0.016043,0.016043,2.679581,1.876967,-1.642897,...,0.56,2.15,-0.02,0.07,-0.56,-0.24,-0.17,2.62,2.25,1.77
7,2010-01-07,1141.689941,0.059932,0.004001,0.000546,0.003116,0.016043,2.679581,1.876967,-1.642897,...,-0.72,0.94,0.50,0.40,0.58,0.58,0.54,-1.85,0.22,-0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2010-05-18,1120.800049,0.041858,-0.014196,0.001109,-0.018800,-0.012145,-3.027366,-4.498971,-5.314639,...,-1.52,0.11,-1.74,-1.39,0.00,1.08,1.11,-0.56,0.37,-0.17
139,2010-05-19,1115.050049,0.096415,-0.005130,-0.014196,0.001109,-0.018800,0.366170,-4.358972,-6.405279,...,-0.76,-4.05,-1.52,-0.78,-0.05,-0.88,-0.88,0.78,-4.06,-2.45
140,2010-05-20,1071.589966,0.230981,0.000546,-0.005130,-0.014196,0.001109,0.366170,-5.013523,0.924679,...,-2.00,-2.19,0.13,0.06,-0.30,-0.96,-0.96,-0.77,-3.08,-0.73
141,2010-05-21,1087.689941,-0.345370,0.015024,0.000538,-0.005130,-0.014196,-4.225672,-2.087540,0.924679,...,1.65,-0.36,1.37,1.36,0.06,-0.20,-0.22,1.07,0.11,-0.50


In [ ]:
df5.to_csv('Cleaned&Processed_S&P.csv')

# Data Integration

Integrate all the dataframes into a single dataset and perform Data Cleaning steps again.

In [ ]:
# Create a list of dataframes
dfs = [df1, df2, df3, df4, df5]

# Concatenate them
df_combined = pd.concat(dfs, ignore_index=True)
print(df_combined.head())

        Date         Close    Volume       mom      mom1      mom2      mom3  \
0 2009-12-31  10428.049805  0.515598  0.014951  0.014951  0.014951  0.014951   
1 2010-01-04  10583.959961  0.515598  0.014951  0.014951  0.014951  0.014951   
2 2010-01-05  10572.019531  0.515598 -0.001128  0.014951  0.014951  0.014951   
3 2010-01-06  10573.679688  0.515598  0.000157 -0.001128  0.014951  0.014951   
4 2010-01-07  10606.860352 -0.665845  0.003138  0.000157 -0.001128  0.014951   

      ROC_5    ROC_10    ROC_15  ...  RUSSELL-F  S&P-F   CHF  Dollar index-F  \
0  1.823357  1.741463 -2.216996  ...      -1.08  -1.00 -0.11           -0.08   
1  1.823357  1.741463 -2.216996  ...       1.61   1.62 -0.57           -0.59   
2  1.823357  1.741463 -2.216996  ...      -0.20   0.31  0.43            0.03   
3  1.823357  1.741463 -2.216996  ...      -0.02   0.07 -0.56           -0.24   
4  1.823357  1.741463 -2.216996  ...       0.50   0.40  0.58            0.58   

   Dollar index  wheat-F   XAG   XAU  

In [ ]:
df_combined['Date'] = pd.to_datetime(df_combined['Date'])
df_combined = df_combined.set_index('Date')
df_combined = df_combined.sort_index()
df_combined.reset_index(inplace=True)
print(df_combined.head())

        Date         Close    Volume       mom      mom1      mom2      mom3  \
0 2009-12-31  10428.049805  0.515598  0.014951  0.014951  0.014951  0.014951   
1 2009-12-31   1115.099976 -0.001074  0.016043  0.016043  0.016043  0.016043   
2 2009-12-31   2269.149902  0.000000  0.017306  0.017306  0.017306  0.017306   
3 2009-12-31   7184.959961 -0.001183  0.019733  0.019733  0.019733  0.019733   
4 2009-12-31    625.390015 -0.001074  0.023521  0.023521  0.023521  0.023521   

      ROC_5    ROC_10    ROC_15  ...  RUSSELL-F  S&P-F   CHF  Dollar index-F  \
0  1.823357  1.741463 -2.216996  ...      -1.08   -1.0 -0.11           -0.08   
1  2.679581  1.876967 -1.642897  ...      -1.08   -1.0 -0.11           -0.08   
2  2.116212  0.830271 -2.571441  ...      -1.08   -1.0 -0.11           -0.08   
3  3.345741  2.391525 -1.556447  ...      -1.08   -1.0 -0.11           -0.08   
4  3.065284  2.009947 -1.164078  ...      -1.08   -1.0 -0.11           -0.08   

   Dollar index  wheat-F  XAG   XAU   

In [ ]:
# Check for missing values in each column
missing_values = df_combined.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

GSPC    5931
IXIC    1977
RUT     1977
NYSE    1977
DJI     1977
S&P     5931
dtype: int64


In [ ]:
for col in df_combined.columns:
    if col != 'Date':
      if pd.api.types.is_numeric_dtype(df_combined[col]):
          df_combined[col] = df_combined[col].fillna(df_combined[col].mean())

In [ ]:
# Check for missing values in each column
missing_values = df_combined.isnull().sum()

# Display columns with missing values
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [ ]:
df_combined.to_csv('Cleaned&Processed_Data.csv')